In [97]:
# import required packages
import random
import os
import sounddevice as sd
from scipy.io.wavfile import write
import wavio as wv
import pandas as pd
import librosa
import re
import numpy as np
import joblib
import ast

In [98]:
# generate the random words
words = ['Environment', 'Archives', 'Pronounciation', 
         'Hour', 'Wednesday', 'Violence', 'Tomb', 'Suite', 
         'Iron', 'Reciept', 'Chores']
random_words = random.sample(words, 5)
random_string = ' '.join(random_words)
print(random_string)

# Directory path to traverse
directory_path = 'models/'
models = []

for word in random_words:
    for file in os.listdir(directory_path):
        if word in file:
            models.append(file)

Pronounciation Wednesday Iron Environment Tomb


In [99]:
models

['Pronounciation_RandomForest_model.joblib',
 'Wednesday_RandomForest_model.joblib',
 'Iron_RandomForest_model.joblib',
 'Environment_RandomForest_model.joblib',
 'Tomb_RandomForest_model.joblib']

In [100]:
# collect audio samples
name = 'Unknown'
freq = 44100
duration = 2

def delete_old_files(folder):
    for filename in os.listdir(folder):
        file_path = os.path.join(folder, filename)
        try:
            if os.path.isfile(file_path):
                os.unlink(file_path)
        except Exception as e:
            print(e)

delete_old_files("audio_files/test")

words = random_words

for word in words:
    for i in range(1):
        file_name = "audio_files/test/" + name + str(i+1) + word + '.wav'
        print("Recording file " + file_name)
        print(f'{word} - {i+1}')
        recording = sd.rec(int(duration * freq), samplerate=freq, channels=2)
        sd.wait()
        write(file_name, freq, recording)
        print("Recorded file " + file_name)

Recording file audio_files/test/Unknown1Pronounciation.wav
Pronounciation - 1
Recorded file audio_files/test/Unknown1Pronounciation.wav
Recording file audio_files/test/Unknown1Wednesday.wav
Wednesday - 1
Recorded file audio_files/test/Unknown1Wednesday.wav
Recording file audio_files/test/Unknown1Iron.wav
Iron - 1
Recorded file audio_files/test/Unknown1Iron.wav
Recording file audio_files/test/Unknown1Environment.wav
Environment - 1
Recorded file audio_files/test/Unknown1Environment.wav
Recording file audio_files/test/Unknown1Tomb.wav
Tomb - 1
Recorded file audio_files/test/Unknown1Tomb.wav


In [101]:
# generate df for test set
# generate the MFCC from audio files - 

# Function to extract MFCC features from an audio file
def extract_mfcc(file_path):
    y, sr = librosa.load(file_path)
    mfccs = librosa.feature.mfcc(y=y, sr=sr)
    return mfccs.tolist()  # Convert to list for easier storage in DataFrame

# Directory containing audio files
directory_path = "audio_files/test"

# Target folder to save DataFrames as CSV files
target_folder = "df/test"

# Create the target folder if it doesn't exist
os.makedirs(target_folder, exist_ok=True)

# List of words
words = random_words

# Initialize a dictionary to store DataFrames for each word
word_dataframes = {word: pd.DataFrame() for word in words}

# Iterate over the filenames
for filename in os.listdir(directory_path):
    # Use regular expression to extract name and word
    match = re.match(r'([A-Za-z]+)(\d+)([A-Za-z]+)', filename)
    if match:
        name = match.group(1)
        word = match.group(3)
        num = int(match.group(2))
        
        # Extract MFCC features
        mfccs = extract_mfcc(os.path.join(directory_path, filename))

        # Create a DataFrame for each person
        person_df = pd.DataFrame({"Name": [name], "MFCC": [mfccs]})
        
        # Concatenate with existing DataFrame for the word
        word_dataframes[word] = pd.concat([word_dataframes[word], person_df], ignore_index=True)

# Save each DataFrame to a separate CSV file in the target folder
for word, df in word_dataframes.items():
    output_file_path = os.path.join(target_folder, f"{word}_dataframe.csv")
    df.to_csv(output_file_path, index=False)
    print(f"DataFrame for word '{word}' saved to '{output_file_path}'")

DataFrame for word 'Pronounciation' saved to 'df/test\Pronounciation_dataframe.csv'
DataFrame for word 'Wednesday' saved to 'df/test\Wednesday_dataframe.csv'
DataFrame for word 'Iron' saved to 'df/test\Iron_dataframe.csv'
DataFrame for word 'Environment' saved to 'df/test\Environment_dataframe.csv'
DataFrame for word 'Tomb' saved to 'df/test\Tomb_dataframe.csv'


In [102]:
# predict from the df

# List of words
words = random_words

model = models
users = []

for i in range(len(words)):
    # Load the dataframe for the current word
    df = pd.read_csv("df/test/" + f"{words[i]}_dataframe.csv")
    # Convert the string representation of MFCC values back to list of lists
    df["MFCC"] = df["MFCC"].apply(ast.literal_eval)
    # Flatten the MFCC values
    df["MFCC"] = df["MFCC"].apply(lambda x: [item for sublist in x for item in sublist])
    loaded_model = joblib.load("models/" + model[i])
    
    X = np.array(df["MFCC"].tolist())
    user = loaded_model.predict(X)
    users.append(user)

users

[array(['Abhishek'], dtype=object),
 array(['Arun'], dtype=object),
 array(['Arun'], dtype=object),
 array(['Sunamdha'], dtype=object),
 array(['Abhishek'], dtype=object)]